<a href="https://colab.research.google.com/github/Teywa-OZIOL/AI_Content_Factory/blob/main/Documentary_Generator/code_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function Installation

In [27]:
!pip install google.genai
!pip install groq

# Import Package

In [28]:
from google import genai
import wave
from google.colab import userdata
from groq import Groq
import json
import re

# Initial Parameters

In [29]:
!mkdir outputs

mkdir: cannot create directory ‘outputs’: File exists


In [30]:
TOPIC = "La guerre froide"
FILENAME = re.sub(r'[^a-zA-Z0-9_-]+', '_', TOPIC.lower())
OUTPUT_DIR = "outputs"

# Part 1 : Plan Generation

In [31]:
def parse_output(text):
        cleaned = text.strip("```python").strip("```").strip()
        try:
            return eval(cleaned)
        except Exception:
            try:
                return json.loads(cleaned)
            except Exception:
                return None

def topic_analyzer(topic):
    GROQ_API_KEY = userdata.get('GROQ')
    client = Groq(api_key=GROQ_API_KEY)

    system_prompt = """
        Tu es un expert en storytelling documentaire et en référencement YouTube (SEO).
        Ton rôle :
        - Créer des plans de documentaires captivants et informatifs.
        - Générer des titres accrocheurs (type YouTube) et des descriptions SEO de 150–250 mots.
        - Employer un ton immersif, pédagogique et crédible.
        - Utiliser des mots-clés pertinents pour le référencement SEO.
        - Toujours respecter strictement le format demandé (dictionnaire Python).
    """

    example = """Exemple de format attendu :
        {
          "titre_video": "La Guerre Froide : Quand le monde a failli exploser",
          "description_SEO": "Découvrez l’histoire fascinante de la Guerre Froide...",
          "mots_cles": ["guerre froide", "URSS", "États-Unis", "crise de Cuba"],
          "chapitres": [
            {"titre": "1. Le monde après 1945", "contenu": "Après la Seconde Guerre mondiale..."},
            {"titre": "2. Le rideau de fer", "contenu": "La division de l’Europe en deux blocs..."},
            ...
          ]
        }
    """

    user_prompt = f"""
        Crée un plan complet de documentaire de 10 à 15 minutes sur : "{topic}".

        Respecte exactement la structure suivante :
        {{
          "titre_video": str,          # titre accrocheur, format YouTube
          "description_SEO": str,      # 150–250 mots, riche en mots-clés, ton narratif
          "mots_cles": [str, ...],     # mots-clés pertinents pour le SEO
          "chapitres": [               # 3 à 6 chapitres, chacun 3 à 5 phrases
            {{"titre": str, "contenu": str}},
            ...
          ]
        }}

        {example}

        Règles :
        - Retourne UNIQUEMENT le dictionnaire Python, sans texte autour.
        - Le ton doit être immersif et dynamique.
    """

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.7,
        max_tokens=2000,
    )

    raw_output = response.choices[0].message.content.strip()

    plan = parse_output(raw_output)

    output_path = f"{OUTPUT_DIR}/PART1_{FILENAME}.json"

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(plan, f, indent=2, ensure_ascii=False)

    print('===============================================')
    print(f"✅ PART 1 : Documentary Plan Generated !!!")
    print('===============================================')

    return

# Part 2 : Script Generation

In [32]:
def generate_text(client, prompt, max_tokens=1500, temperature=0.7):
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        text = response.choices[0].message.content.strip()
        return text.strip("```").strip("python").strip()

def script_generator(topic):

    GROQ_API_KEY = userdata.get('GROQ')
    client = Groq(api_key=GROQ_API_KEY)

    plan_path = f"{OUTPUT_DIR}/PART1_{FILENAME}.json"

    with open(plan_path, "r", encoding="utf-8") as f:
        plan = json.load(f)

    topic = plan["titre_video"]

    output_json_path = f"{OUTPUT_DIR}/PART2_{FILENAME}.json"

    script_chapters = []

    intro_prompt = f"""
        Rédige une introduction captivante pour le documentaire '{topic}'.
        Durée : ~250 mots (~2 min narration).
        Style : narratif immersif, comme une voix-off de Netflix/Arte.
        Ne mentionne aucun chapitre encore. Ajoute entre crochet,
        la manière dont il faut raconter certaines phrases entre colère, joie, tristesse, résignation, ...
        ajoute aussi des rires, des soupires, des exclamations, des petits défauts authentique
    """

    intro_text = generate_text(client, intro_prompt)

    script_chapters.append({
        "numero": 0,
        "titre": "Introduction",
        "texte": intro_text
    })

    print("✅ PART 2 : Introduction Script Generated")

    previous_context = intro_text
    for idx, chap in enumerate(plan["chapitres"], start=1):
        chap_prompt = f"""
            Tu écris le chapitre {idx} d'un documentaire sur '{topic}'.
            Titre du chapitre : "{chap['titre']}"
            Résumé du plan : "{chap['contenu']}"
            Contexte narratif précédent : "{previous_context}"

            Écris un texte narratif immersif, détaillé (~350 mots minimum), fluide et captivant.
            Inclure anecdotes, dates, personnages et transitions.
            Ne contredis aucun texte précédent.
            Ajoute entre crochet,
            la manière dont il faut raconter certaines phrases entre colère, joie, tristesse, résignation, ...
            ajoute aussi des rires, des soupires, des exclamations, des petits défauts authentique
        """
        chap_text = generate_text(client, chap_prompt)
        script_chapters.append({
            "numero": idx,
            "titre": chap["titre"],
            "texte": chap_text
        })
        previous_context += " " + chap_text

        print(f"✅ PART 2 : {idx} Script Generated")

    conclusion_prompt = f"""
        Rédige une conclusion forte et mémorable pour le documentaire '{topic}'.
        Contexte narratif précédent : "{previous_context}"
        Durée : ~250 mots (~2 min narration).
        Résume l'ensemble, relie les chapitres et termine sur une phrase marquante.
        Ajoute entre crochet,
        la manière dont il faut raconter certaines phrases entre colère, joie, tristesse, résignation, ...
        ajoute aussi des rires, des soupires, des exclamations, des petits défauts authentique
    """

    conclusion_text = generate_text(client, conclusion_prompt)

    script_chapters.append({
        "numero": len(plan["chapitres"]) + 1,
        "titre": "Conclusion",
        "texte": conclusion_text
    })
    print("✅ PART 2 : Conclusion Script Generated")

    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(script_chapters, f, indent=2, ensure_ascii=False)

    print('===============================================')

    return

# Part 3 : Audio Generation

In [33]:
GOOGLE_API_KEY = userdata.get('GEMINI')
client = genai.Client(api_key=GOOGLE_API_KEY)

prompt = """
    Read this in French with the voice of an old sage recounting the war:
    C'était en 1939, les français sont attaqués par les Allemands. Ce fut cruel, apre, et ça a bouleversé nos vies.
    Then, you sigh and say with great emotion :
    Je me souviens des pleures de ma mère qui me regardait partir au front, c'était la guerre
    Then, You laught and you say :
    Mais on les a bien défoncé
"""

def generate_audio(client, prompt):
  voice_name = "Iapetus"
  MODEL_ID = "gemini-2.5-flash-preview-tts"
  response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config={"response_modalities": ['Audio'],
            "speech_config": {"voice_config": {"prebuilt_voice_config": {"voice_name": voice_name}}}},
    )
  data = response.candidates[0].content.parts[0].inline_data.data
  return data

def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

data = generate_audio(client, prompt)
wave_file('out.wav', data)

'\nGOOGLE_API_KEY = userdata.get(\'GEMINI\')\nclient = genai.Client(api_key=GOOGLE_API_KEY)\n\nprompt = \n    Read this in French with the voice of an old sage recounting the war:\n    C\'était en 1939, les français sont attaqués par les Allemands. Ce fut cruel, apre, et ça a bouleversé nos vies.\n    Then, you sigh and say with great emotion :\n    Je me souviens des pleures de ma mère qui me regardait partir au front, c\'était la guerre\n    Then, You laught and you say :\n    Mais on les a bien défoncé\n\n\ndef generate_audio(client, prompt):\n  voice_name = "Iapetus"\n  MODEL_ID = "gemini-2.5-flash-preview-tts"\n  response = client.models.generate_content(\n    model=MODEL_ID,\n    contents=prompt,\n    config={"response_modalities": [\'Audio\'],\n            "speech_config": {"voice_config": {"prebuilt_voice_config": {"voice_name": voice_name}}}},\n    )\n  data = response.candidates[0].content.parts[0].inline_data.data\n  return data\n\ndef wave_file(filename, pcm, channels=1, ra

# MAIN

In [34]:
#from google.colab import files
#files.download("mon_fichier.csv")

In [35]:
def main(topic):

    topic_analyzer(topic)
    script_generator(topic)
    #voice_generation()
    #image_generation()
    #video_generation()
    #music_generation()
    #video_editing()

    return

main(TOPIC)

✅ Plan documentaire sauvegardé dans : outputs/PART1_la_guerre_froide.json
{
  "titre_video": "La Guerre Froide : Les 50 ans qui ont failli détruire le monde",
  "description_SEO": "Plongez dans l'histoire captivante de la Guerre Froide, une période de tensions géopolitiques qui a mis le monde au bord de la catastrophe nucléaire. Découvrez comment les superpuissances que sont les États-Unis et l'URSS ont joué un jeu de chat et de souris, créant des alliances, des blocs et des crises qui ont façonné le monde moderne. De la division de l'Allemagne à la crise de Cuba, revivez les moments clés de cette époque troublée et découvrez comment la Guerre Froide a laissé une empreinte durable sur la politique, la société et la culture internationales. Avec des archives inédites et des analyses expertes, ce documentaire vous transporte au cœur de la Guerre Froide, vous permettant de comprendre les complexités et les conséquences de ce chapitre crucial de l'histoire du 20e siècle. Les mots-clés de c